In [1]:
# Imports and plotting setups
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import math

import sys

import math_funcs

from scipy.special import loggamma, factorial, gamma
from decimal import Decimal
import decimal

from antenna_patterns import uhf_antenna, hf_antenna
from transmission_reflection_coefficients import transmissivity_fBm, reflectivity_fBm
from T_B_Europan_sky import angle_grid, T_B_sub_jovian_point, T_B_anti_jovian_point, T_B_anti_orbital_point

from scipy.constants import (
    epsilon_0,   # Permittivity of free space (vacuum)
    mu_0,        # Permeability of free space (vacuum)
    c,           # Speed of light in vacuum
    #e,           # Elementary charge
    #h,           # Planck constant
    #hbar,        # Reduced Planck constant (h-bar)
    k,           # Boltzmann constant
    #G,           # Newtonian constant of gravitation
    #m_e,         # Electron mass
    #m_p,         # Proton mass
    #m_n,         # Neutron mass
    #alpha,       # Fine-structure constant
    eV,          # Electron volt
)

In [ ]:
class europa_ice_model:
    def __init__(self,
        T_u = 104, #K
        T_l = 230, #K
        T_melt = 273.13, #K
        T_conv = 251.6, #K
        D_cond = 10.4e3, #m
        D_phi = 3.2e3, #m
        eta_vac = 0.1, #np.arrange(0, 0.3, 0.1)
        rho_salt = 1e-5, #np.linspace(1e-3, 4.2e-2, 10)
        D_conv = 5.8e3, #m
        delta_d = 10 #m
        ):

        self.T_u = T_u 
        self.T_l = T_l 
        self.T_melt = T_melt 
        self.T_conv = T_conv 
        self.D_cond = D_cond 
        self.D_phi = D_phi 
        self.eta_vac = eta_vac 
        self.rho_salt = rho_salt 
        self.D_conv = D_conv 
        self.delta_d = delta_d
        
        self.D_total = self.D_cond + self.D_conv

        self.cryosphere_model_df = pd.DataFrame({'Depth (m)':\
            np.arange(0, self.D_total+delta_d, delta_d)})
        self.cryosphere_model_df['Temperature (K)'] = \
            self.cryosphere_model_df['Depth (m)'].map(self.temperature_at_depth)
        self.cryosphere_model_df['Porosity (m^3/m^3)'] = \
            self.cryosphere_model_df['Depth (m)'].map(self.porosity_at_depth)
        self.cryosphere_model_df['Salt fraction (kg/kg)'] = \
            self.cryosphere_model_df['Depth (m)'].map(self.salt_fraction_at_depth)

        # Estimate the dielectric constant of the ice
        T = self.cryosphere_model_df['Temperature (K)']

        self.cryosphere_model_df['epsilon_s_prime'] = 3.1884 + 0.00091*(T - 273.13)
        self.cryosphere_model_df['epsilon_s_primeprime'] = 10**(-3.0129 + 0.0123*(T - 273.13))
        self.cryosphere_model_df['unadjusted epsilon_s_primeprime'] = self.cryosphere_model_df['epsilon_s_primeprime']

        # Estimate the conductivity of the ice
        molar_mass_salt = (35.453 + 22.990) / 1000 # kg/mol
        molarity_salt = self.cryosphere_model_df['Salt fraction (kg/kg)'] / molar_mass_salt
        micro_molarity_salt = molarity_salt * 1e6

        self.cryosphere_model_df['sigma_s'] = 1e-6 * \
            (9 * np.e ** ((0.58*eV/k) * (1 / 258.15 - 1 / T)) \
            + 0.55 * micro_molarity_salt * np.e ** ((0.22*eV/k) * (1 / 258.15 - 1 / T)))

        self.cryosphere_model_df['unadjusted sigma_s'] = self.cryosphere_model_df['sigma_s']

        # Modify the epsilon s primes by the Maxwell-Garnett
        epsilon_s_prime = self.cryosphere_model_df['epsilon_s_prime']
        epsilon_s_primeprime = self.cryosphere_model_df['epsilon_s_primeprime']
        eta_vac = self.cryosphere_model_df['Porosity (m^3/m^3)']
        epsilon_s = (epsilon_s_prime - 1j * epsilon_s_primeprime) * epsilon_0
        epsilon_s_but_with_sigma_loss =  (epsilon_s_prime - 1j * epsilon_s_primeprime) * epsilon_0
        epsilon_m = epsilon_s + (3 * eta_vac * epsilon_s * (epsilon_0 - epsilon_s)) \
            / ((2 * epsilon_s + epsilon_0) - eta_vac * (epsilon_0 - epsilon_s))

        self.cryosphere_model_df['epsilon_s_prime'] = np.real(epsilon_m / epsilon_0)
        self.cryosphere_model_df['epsilon_s_primeprime'] = -1 * np.imag(epsilon_m / epsilon_0)

        



    def temperature_at_depth(self, d):
        if d > self.D_cond + self.D_conv:
            return self.T_melt
        elif d > self.D_cond:
            return self.T_conv
        else:
            m, b = math_funcs.linear_fit(
                0, self.T_u, 
                self.D_cond, self.T_l)
            return m * d + b

    def porosity_at_depth(self, d):
        if d <= self.D_phi:
            return self.eta_vac
        else:
            return 0
        
    def salt_fraction_at_depth(self, d):
        return self.rho_salt

europa_ice_model().cryosphere_model_df

,Depth (m),Temperature (K),Porosity (m^3/m^3),Salt fraction (kg/kg),epsilon_s_prime,epsilon_s_primeprime,sigma_s
0,0.0,104.000000,0.1,0.00001,2.779818,0.000007,4.050878e-11
1,10.0,104.121154,0.1,0.00001,2.779914,0.000007,4.168254e-11
2,20.0,104.242308,0.1,0.00001,2.780009,0.000007,4.288747e-11
3,30.0,104.363462,0.1,0.00001,2.780104,0.000007,4.412431e-11
4,40.0,104.484615,0.1,0.00001,2.780200,0.000007,4.539383e-11
...,...,...,...,...,...,...,...
1616,16160.0,251.600000,0.0,0.00001,3.168808,0.000528,7.731257e-05
1617,16170.0,251.600000,0.0,0.00001,3.168808,0.000528,7.731257e-05
1618,16180.0,251.600000,0.0,0.00001,3.168808,0.000528,7.731257e-05
1619,16190.0,251.600000,0.0,0.00001,3.168808,0.000528,7.731257e-05


In [12]:
f = 100e6
epsilon_s_prime = 3.2
epsilon_s_primeprime = 1e-10
sigma_s = 1e-10
epsilon_s = (3.2 - 1j * epsilon_s_primeprime) * epsilon_0
epsilon_s_but_with_sigma_loss =  (3.2 - 1j * sigma_s) * epsilon_0
epsilon_s_but_with_both_losses =  (3.2 - 1j *  epsilon_s_primeprime - 1j * sigma_s / (epsilon_0 * 2 * np.pi * f)) * epsilon_0
def maxwell_garnett_mixing_formula(epsilon_s, eta_vac):
    return epsilon_s + (3 * eta_vac * epsilon_s * (epsilon_0 - epsilon_s)) \
    / ((2 * epsilon_s + epsilon_0) - eta_vac * (epsilon_0 - epsilon_s))

maxwell_garnett_mixing_formula(0.1, epsilon_s), \
    maxwell_garnett_mixing_formula(0.1, epsilon_s_but_with_sigma_loss), \
    maxwell_garnett_mixing_formula(0.1, epsilon_s_but_with_both_losses)


((0.09999999999574999+1.3281281717059776e-22j),
 (0.09999999999574999+1.3281281717059776e-22j),
 (0.09999999999574999+2.4006054277107815e-20j))